In [27]:
import pandas as df
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange

import os
import sys
from pathlib import Path

os.chdir(Path(sys.path[0]).parent)
import modules.utils as utils

import multiprocessing as mp
import time

In [24]:
def random_square(seed):
    np.random.seed(seed)
    random_num = np.random.randint(0, 10)
    return random_num**2

In [ ]:
t0 = time.time()
results = []
for i in trange(1000000): 
    results.append(random_square(i))
t1 = time.time()
print(f'Execution time {t1 - t0} s')

In [33]:
t0 = time.time()
n_cpu = mp.cpu_count()

pool = mp.Pool(processes=n_cpu)
results = [pool.map(random_square, trange(1000000))]
t1 = time.time()
print(f'Execution time {t1 - t0} s')

KeyboardInterrupt: 

In [7]:
Directory = "data"
L = 32

u_ds = utils.concatenate_time(Directory, 'u')
v_ds = utils.concatenate_time(Directory, 'v')
w_ds = utils.concatenate_time(Directory, 'w')
theta_ds = utils.concatenate_time(Directory, 'theta')
assert u_ds.shape == v_ds.shape == w_ds.shape == theta_ds.shape, 'u,v,w,theta have different shape'

u_coarse = utils.coarse_array(u_ds, L)
v_coarse = utils.coarse_array(v_ds, L)
w_coarse = utils.coarse_array(w_ds, L)

wtheta_ds = w_ds*theta_ds
tke_ds = utils.coarse_array(u_ds*u_ds, L) - u_coarse*u_coarse + utils.coarse_array(v_ds*v_ds, L) - v_coarse*v_coarse + utils.coarse_array(w_ds*w_ds, L) - w_coarse*w_coarse
tke_in = utils.variable_samples(tke_ds, 1)
output_ds = utils.variable_samples(wtheta_ds, L)

variables = ['u', 'v', 'w', 'theta']  # add 's' 
input_ds = utils.input_dataset(Directory, variables, L)
tot_ds = np.concatenate((np.concatenate((input_ds,tke_in), axis=1), output_ds), axis=1)

variables.append('tke')
variables.append('wtheta')
print(variables)

utils.write_nc_file(tot_ds,str(L),variables,4)

['u', 'v', 'w', 'theta', 'tke', 'wtheta']
writing out


In [8]:
path_data = 'data/input_dataset_L_32.nc'
ds_init = xr.open_dataset(path_data)
df_init = ds_init.to_dataframe()
df_init.head

<bound method NDFrame.head of              sample
index i            
0     0   14.980471
      1   14.990431
      2   15.000865
      3   15.009308
      4    0.500632
...             ...
255   19   0.875857
      20   0.132721
      21   0.138177
      22   0.143849
      23   0.149717

[6144 rows x 1 columns]>

In [13]:
nz=4
len_samples = nz*len(variables)
tot_ds = df_init.to_numpy()
n_samples = len(tot_ds)//len_samples
tot_ds = tot_ds.reshape(n_samples, len_samples)
tot_ds.shape

(256, 24)